In [ ]:
#dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import pymongo
import time
import numpy as np
import requests
import pandas as pd


In [ ]:
browser = Browser('chrome')

In [ ]:
def get_soup(url):
    browser.visit(url)
    html = browser.html    
    soup = bs(html, 'html.parser')
    return soup

In [ ]:
#scraping top ten tags. 
#flask /top10tags
soup = get_soup('http://quotes.toscrape.com/')
sidebar = soup.find_all('span', class_='tag-item')
item = sidebar[0]
tags = []

for item in sidebar:
    tag = item.a.getText()
    tags.append(tag)
    print(tag)

In [ ]:
#quotes, authors, tags 
#looping through EACH page
quote_all = []
quote_tbl1 = []
pages = np.arange(1,11,1)#start page 1, stop page 10, step 1 

for page in pages:
    page = requests.get('http://quotes.toscrape.com/page/' +str(page)+ '/')
    soup = bs(page.text,'html.parser')
    
    quotes = soup.find_all('span', class_='text')  
    authors = soup.find_all('small', class_='author')
    tags = soup.find_all('div', class_="tags")

    
    for i in range(0,len(quotes)):
        q = quotes[i].text
        a = authors[i].text
        
        
        
        tags = soup.find_all('div', class_='tags')
        quotetags = tags[i].find_all('a',class_='tag')

        t = [ ] 
        
        for tags in quotetags:
            taggers = tags.text
            t.append(taggers)
        
        
        #place into dictionary 
        data = {'quote:': q,
               'author name:': a,
               'tags' : t
               }
        quote_all.append(data)
        tbl1 = {'author_name': a,
               'quote': q
               }
        quote_tbl1.append(tbl1)

   
        
    #print(quote_all);


In [ ]:
#quote_all
quote_tbl1

In [ ]:
#get author date of birth, description
author_all = [ ] 

pages = np.arange(1,11,1)#start page 1, stop page 10, step 1 

for page in pages:
    page = requests.get('http://quotes.toscrape.com/page/' +str(page)+ '/')
    soup = bs(page.text,'html.parser')

    for link in soup.find_all('a',href=True):
        if link.get_text() == "(about)":
            author_url = 'http://quotes.toscrape.com' + link["href"]
            author_info = requests.get('http://quotes.toscrape.com' + link["href"])
            soup = bs(author_info.text,'html.parser')

            author_born = soup.find_all('span', class_='author-born-date')
            author_description = soup.find_all('div', class_='author-description')
            data = {"DOB:": author_born[0].text,
                   "Description:": author_description[0].text
                   }
            author_all.append(data)
            
            #print('Author URL : http://quotes.toscrape.com' + link["href"])
            #print(f'Author born: {author_born[0].text}, Author Description: {author_description[0].text}')
        else:
            continue




In [ ]:
#author_all

In [ ]:
# Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

#### - one table for quotes, this table can have the quote and author relationship

In [ ]:
#Load Quotes data
db = client.quote_etl
quote = db.author_quote
quote.insert_many(quote_tbl1)

#### - one table for author information

In [ ]:
#Load Author Data
db = client.quote_etl
author = db.authors
author.insert_many(author_all)